In [3]:
from selenium import webdriver
from bs4 import BeautifulSoup
import re

from time import sleep
import pandas as pd

import numpy as np
import numpy as np

import pandas as pd
import numpy as np

In [2]:
base_url = "https://www.nepremicnine.net/oglasi-prodaja/ljubljana-mesto/stanovanje/2-sobno,2.5-sobno,3-sobno,3.5-sobno,4-sobno,4.5-sobno/cena-od-220000-do-300000-eur,velikost-od-50-do-200-m2"
sort_suffix = '?s=16'

In [3]:
def entry_parser(entry):
    meta_info = dict(
        url = entry.find('a', {'class': 'url-title-d'})['href'],
        title = entry.find('a', {'class': 'url-title-d'})['title'].lower(),
        price = float(entry.find('meta', {'itemprop': 'price'})['content']),
        **dict(zip(['area_m2', 'year', 'floor'],  [item.text for item in entry.find_all('li')])),
        category = re.search('Prodaja: (\w+)', str(entry)).group(1).lower(),
        room_type = re.search('[\d,]+-sobno', str(entry)).group()
    )
    meta_info['area_m2'] = float(re.search('[\d,]+', meta_info['area_m2']).group().replace(',', '.'))
    meta_info['year'] = int(re.search('[\d]+', meta_info['year']).group())
    return meta_info

In [5]:
entries = []

page_idx = 1
while True:
    # wait a bit
    sleep(np.random.randint(10, 19) / 10)

    # set a headless driver
    chrome_options = webdriver.ChromeOptions()
    chrome_options.add_argument("--headless")

    # set the user-agent back to chrome.
    user_agent = (
        "Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/60.0.3112.50 Safari/537.36"
    )
    chrome_options.add_argument(f"user-agent={user_agent}")

    driver = webdriver.Chrome(options=chrome_options)
    driver.set_window_size(1080, 800)  # set the size of the window

    # get page content
    print(f"Parsing page: {page_idx}", end=" ... ")
    driver.get(f"{base_url}/{page_idx}/{sort_suffix}")

    # parse ads
    soup_entries = BeautifulSoup(driver.page_source, "lxml").find_all("div", {"class": "property-details"})
    print(f"[Found {len(soup_entries)} ads]")

    if len(soup_entries) == 0:
        print("\tFinished parsing content!")
        break

    entries.extend([entry_parser(entry) for entry in soup_entries])
    page_idx += 1

Parsing page: 1 ... [Found 20 ads]
Parsing page: 2 ... [Found 20 ads]
Parsing page: 3 ... [Found 20 ads]
Parsing page: 4 ... [Found 20 ads]
Parsing page: 5 ... [Found 20 ads]
Parsing page: 6 ... [Found 20 ads]
Parsing page: 7 ... [Found 20 ads]
Parsing page: 8 ... [Found 20 ads]
Parsing page: 9 ... [Found 20 ads]
Parsing page: 10 ... [Found 13 ads]
Parsing page: 11 ... [Found 0 ads]
	Finished parsing content!


In [18]:
found_entries

,url,title,price,area_m2,year,floor,category,room_type
0,https://www.nepremicnine.net/oglasi-prodaja/ze...,zelena jama,260000.0,69.00,1960,1/4,stanovanje,"2,5-sobno"
1,https://www.nepremicnine.net/oglasi-prodaja/lj...,"lj. vič, bližina centra, mirna lokacija",269000.0,72.80,1962,3/4,stanovanje,3-sobno
2,https://www.nepremicnine.net/oglasi-prodaja/st...,štepanjsko naselje,239000.0,60.10,1975,1/4,stanovanje,2-sobno
3,https://www.nepremicnine.net/oglasi-prodaja/ze...,zelena jama,260000.0,71.90,1960,1/4,stanovanje,3-sobno
4,https://www.nepremicnine.net/oglasi-prodaja/do...,dolgi most,299705.2,77.01,2023,1/2,stanovanje,3-sobno
...,...,...,...,...,...,...,...,...
188,https://www.nepremicnine.net/oglasi-prodaja/ro...,"rožna dolina, lj. vič postojnska ul. 25",249900.0,67.80,1963,4/4,stanovanje,"2,5-sobno"
189,https://www.nepremicnine.net/oglasi-prodaja/tr...,trnovo,280000.0,58.00,1981,5/7,stanovanje,2-sobno
190,https://www.nepremicnine.net/oglasi-prodaja/lj...,"lj. center, obnovljeno polkletno stanovanje z ...",220000.0,59.60,1895,PK/2,stanovanje,3-sobno
191,https://www.nepremicnine.net/oglasi-prodaja/lj...,lj. center,295000.0,85.30,1935,P,stanovanje,3-sobno


In [7]:
found_entries = pd.DataFrame(entries)
found_entries.to_csv('./nepremicnine_db.csv')